<div style="background-color:#ff0000; color:#ffffff; padding:10px; border-radius:5px;">
    <strong>Warning:</strong> Do not run in vs code!
</div>


<div style="background-color:#ff0000; color:#ffffff; padding:10px; border-radius:5px;">
    <strong>Warning:</strong> Do not run in vs code!
</div>


<div style="background-color:#ff0000; color:#ffffff; padding:10px; border-radius:5px;">
    <strong>Warning:</strong> Do not run in vs code!
</div>


# 0. Load Modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import transforms, datasets
import pandas as pd
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
import os

# 1. Very naive Model

In [ ]:
# Step 1: Prepare the Dataset
class NPZDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.files = [f for f in os.listdir(data_dir) if self._check_file_shape(f)]

    def _check_file_shape(self, file):
        file_path = os.path.join(self.data_dir, file)
        data = np.load(file_path)['arr_0']
        return data.shape == (166, 166, 30)

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = os.path.join(self.data_dir, self.files[idx])
        data = np.load(file_path)['arr_0']
        label = 1 if 'data1' in self.data_dir else 0
        return torch.from_numpy(data).float(), label

# Load datasets
train_data1 = NPZDataset('./data1')
train_data0 = NPZDataset('./data0')

# Combine datasets
train_data = torch.utils.data.ConcatDataset([train_data1, train_data0])

# Create DataLoader
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

# Step 2: Define the Neural Network
class RCNN(torch.nn.Module):
    def __init__(self):
        super(RCNN, self).__init__()
        
        self.input_layer=torch.nn.Linear(in_features=500*500*15, out_features=500*500*15)

        self.norm=torch.nn.BatchNorm2d()#not sure this is sufficent for batch renormalization

        #Three convolutionnal and batch renormalization
        self.conv1=torch.nn.Conv2d(in_channels=15, out_channels=64, kernel_size=3, stride=1) #need padding of one normally but not precised
        self.relu1=torch.nn.ReLU()
        self.norm1=torch.nn.BatchNorm2d()

        self.conv2=torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1)
        self.relu2=torch.nn.ReLU()
        self.norm2=torch.nn.BatchNorm2d()

        self.conv3=torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1)
        self.relu3=torch.nn.ReLU()
        self.norm3=torch.nn.BatchNorm2d()

        #First max pooling
        self.max1=torch.nn.MaxPool2d(kernel_size=2,stride=2)

        #Three convolutionnal and batch renormalization
        self.conv4=torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1)
        self.relu4=torch.nn.ReLU()
        self.norm4=torch.nn.BatchNorm2d()

        self.conv5=torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1)
        self.relu5=torch.nn.ReLU()
        self.norm5=torch.nn.BatchNorm2d()

        self.conv6=torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1)
        self.relu6=torch.nn.ReLU()
        self.norm6=torch.nn.BatchNorm2d()

        #Second maxpool
        self.max2=torch.nn.MaxPool2d(stride=2,kernel_size=2)

        #Flatten
        self.flat1=torch.nn.Flatten()

        #fully connected with dropout
        self.lin=torch.nn.Linear(in_features=2304, out_features=1024)
        self.drop=torch.nn.Dropout(p=0.5)

        #LSTM
        self.lstm=torch.nn.LSTM(input_size=1024,output_size=512, num_layers=1)

        #output
        self.output_layer=torch.nn.Softmax(dim=1)
        
    def forward(self,x):
        x=self.input_layer(x)
        x=self.norm(x)#not sure this is sufficent for batch renormalization

        #Three convolutionnal and batch renormalization
        x=self.conv1(x) 
        x=self.relu1(x)
        x=self.norm1(x)

        x=self.conv2(x)
        x=self.relu2(x)
        x=self.norm2(x)

        x=self.conv3(x)
        x=self.relu3(x)
        x=self.norm3(x)

        #First max pooling
        x=self.max1(x)

        #Three convolutionnal and batch renormalization
        x=self.conv4(x)
        x=self.relu4(x)
        x=self.norm4(x)

        x=self.conv5(x)
        x=self.relu5(x)
        x=self.norm5(x)

        x=self.conv6(x)
        x=self.relu6(x)
        x=self.norm6(x)

        #Second maxpool
        x=self.max2(x)

        #Flatten
        x=self.flat1(x)

        #fully connected with dropout
        x=self.lin(x)
        x=self.drop(x)

        #LSTM
        x=self.lstm(x)

        #output
        x=self.output_layer(x)
        
        return x
    
model = RCNN()

# Step 3: Training Parameters
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), **optimizer_kwargs)
criterion = torch.nn.functional.cross_entropy
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=(len(train_loader.dataset) * num_epochs) // train_loader.batch_size,
)


# Step 4: Training the Network
lr_history = []
train_loss_history = []
train_acc_history = []
val_loss_history = []
val_acc_history = []
for epoch in range(1, num_epochs + 1):
    train_loss, train_acc, lrs = train_epoch(
        model, optimizer, scheduler, criterion, train_loader, epoch, device
    )
    train_loss_history.extend(train_loss)
    train_acc_history.extend(train_acc)
    lr_history.extend(lrs)

    val_loss, val_acc = validate(model, device, val_loader, criterion)
    val_loss_history.append(val_loss)
    val_acc_history.append(val_acc)

# Step 4: Evaluate the Model
# Implement the evaluation on validation set

print('Finished Training')
